In [1]:
import sqlite3
import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')
path = '/content/drive/MyDrive/working'

conn = sqlite3.connect(':memory:')

df = pd.read_csv(f'{path}/reserve.csv')
df.to_sql('reserve_tb', conn, index=False)

query = "SELECT * FROM reserve_tb LIMIT 5"
pd.read_sql(query, conn)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [2]:
query = """

SELECT
  *,
  ROW_NUMBER()
    OVER(PARTITION BY customer_id ORDER BY reserve_datetime) AS log_no
FROM
  reserve_tb
LIMIT
  5

"""

pd.read_sql(query, conn)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5


In [3]:
query = """

SELECT
  hotel_id,
  RANK() OVER (ORDER BY COUNT(*) DESC) AS rsv_cnt_rank
FROM
  reserve_tb
GROUP BY
  hotel_id
LIMIT
  5

"""

pd.read_sql(query, conn)

,hotel_id,rsv_cnt_rank
0,h_241,1
1,h_144,2
2,h_37,3
3,h_142,3
4,h_82,5
